In [ ]:
def rolling_average(data, window_size):
    """
    Computes the rolling average of a 1D array.
    
    Parameters:
    - data: array-like, the input signal.
    - window_size: int, the size of the rolling window.
    
    Returns:
    - np.ndarray, the rolling average (same length as input, with NaNs at the edges if padding is used).
    """
    if window_size < 1:
        raise ValueError("Window size must be at least 1.")
    if len(data) < window_size:
        raise ValueError("Window size must not exceed the length of the data.")
    
    return np.convolve(data, np.ones(window_size)/window_size, mode='same')

signal = data[data['Cell ID'] == PSM_index[10]]['Spot center intensity Center ch1']


# Normalise signal by dividing by the mean  
signal = signal / np.mean(signal)
signal = signal.values
print(signal)
# Smoothen signal with rolling average
window_length_smooth = 5
window_length_trend = 15

signal_smooth = rolling_average(signal, window_length_smooth)
trend = rolling_average(signal_smooth, window_length_trend)
print(signal_smooth)

signal_detrended = signal_smooth - trend
print(signal_detrended)

In [ ]:
def pyBoating(t, signal, sel, Ts, Tc, ws, phi, power_threshold, ws_smooth, plot, power_max):

    # Determine signal to analyse
    if sel == "raw":
        sig = signal
        signals_df = pd.DataFrame({'time': t, 
                                       'signal': signal}) 

    elif sel == "detrended" or sel == "normed":
        # Calculate the trend and detrend the signal
        # print(signal.dtype)
        # signal = signal.values
        # print(signal.values)
        trend = pb.sinc_smooth(signal, T_c=Tc, dt=Ts)
        detrended = signal - trend

        if sel == "normed":
            # Calculate amplitude envelope of the detrended signal
            # Only explicitly calculate to be store in df
            envelope = pb.sliding_window_amplitude(detrended, window_size=ws, dt=Ts)

            # Normalise detrended signal with envelope
            sig = pb.normalize_with_envelope(detrended, window_size=ws, dt=Ts)

            # Save to df
            signals_df = pd.DataFrame({'time': t, 
                                       'signal': signal, 
                                       'trend': trend, 
                                       'detrended': detrended, 
                                       'envelope': envelope,
                                       'normed' : sig})
        else:
            sig=detrended

            # Save to df
            signals_df = pd.DataFrame({'time': t, 
                                       'signal': signal, 
                                       'trend': trend, 
                                       'detrended': detrended})

    # Wavelet transform
    mod, trans = pb.compute_spectrum(sig, dt=Ts, periods=phi)

    # Collect into dataframe
    spectrum_df = pd.DataFrame({
        'time': np.repeat(t, len(phi)),
        'periods': np.tile(periods, len(t)),
        'power': mod.flatten()
    })

    # Fourier transform
    fourier_df = fourier(sig, Ts)

    # Ridge detection
    ridge_df = ridge(t, sig, mod, trans, phi, power_threshold, ws_smooth)

    # # Plot
    # if plot:
    #     fig = _plt(sel, t, sig, phi, mod, ridge_df, power_max)

    #     return signals_df, spectrum_df, fourier_df, ridge_df, fig
    # else:

    return signals_df, spectrum_df, fourier_df, ridge_df

# Compute fourier transform
def fourier(signal, Ts):
    freqs, power = pb.compute_fourier(signal, dt=Ts)
    fourier_df = pd.DataFrame({'frequency':freqs, 'power':power})
    return fourier_df


# Ridge detection
def ridge(t, signal, mod, trans, phi, power_threshold, ws_smooth):

    # Ridge detection
    r = pb.get_maxRidge_ys(mod)
    r_eval = pb.eval_ridge(
        ridge_y=r,
        transform=trans,
        signal=signal,
        periods=phi,
        tvec=t,
        power_thresh=power_threshold,
        smoothing_wsize=ws_smooth
        )
    # Convert r_eval to a DataFrame
    ridge_df = pd.DataFrame(r_eval)

    # Check if ridge_df is empty
    if ridge_df.empty:
        # Add a row with missing values
        ridge_df = ridge_df.append(pd.Series([np.nan] * ridge_df.shape[1], index=ridge_df.columns), ignore_index=True)

    return ridge_df

# function _plt(
#     sel::String,
#     t::Vector{Float64},
#     signal::Vector{Float64},
#     ϕ::Vector{Float64},
#     mod::Matrix{Float64},
#     r::DataFrame,
#     power_max::Union{Missing,Float64}=missing)

#     fig=Figure(
#         resolution=(600,700)
#         )
#     ax_signal = Axis(
#         fig[1, 1],
#         ylabel=sel,
#         limits=(minimum(t), maximum(t), nothing, nothing)
#     )

#     ax_spectrum = Axis(
#         fig[2, 1],
#         xlabel="time",
#         ylabel="period",
#         limits=(minimum(t), maximum(t), nothing, nothing)
#     )

#     # Plot signal
#     scatterlines!(ax_signal, t, signal)

#     # Plot spectrum
#     pwr_max = ismissing(power_max) ? maximum(mod) : power_max
#     hm = heatmap!(
#         ax_spectrum, t, ϕ, mod', colorrange=(0, pwr_max)
#     )
    
#     # Plot ridge
#     scatter!(ax_spectrum, r.time, r.periods, 
#         color=:tomato, markersize=4)
    
#     # Colormap
#     Colorbar(
#         fig[2, 2], hm,
#         label="Wavelet power",
#         flipaxis=true
#         )

#     rowsize!(fig.layout, 2, Relative(2 / 3))
#     colsize!(fig.layout, 1, Relative(1))

#     return fig

# Sampling time (min)
T_sampling = 10.0

# Cut-off for the sinc smoothing (min) ~1.5*expected period
T_cutoff = 225.0

# Window size of amplitude envelope (min) ~3*expected period 
T_window = 450.0

# Period range (min)
period_start = 50.0
period_steps = 1.0
period_stop = 350.0
periods = np.linspace(period_start, period_stop, int((period_stop - period_start) / period_steps) + 1)

# Smoothing window size for ridge
window_ridge = 45

# Power threshold for ridge detection
thresholds_df = pd.DataFrame({
    'Region': ["PSM", "NT", "PSM", "NT"],
    'maxPower': [30.0, 40.0, 30.0, 40.0],
    'p95': [0.000, 0.000, 0.000, 0.000]
})

time = cell_data['Time'].values
signal = cell_data['Spot center intensity Center ch1'].values
thresholds = thresholds_df[(thresholds_df['Region'] == cell_data['Label'].iloc[0])]

pwr_thresh = thresholds['p95'].iloc[0]
pwr_max = thresholds['maxPower'].iloc[0]

signals_df_i, spectrum_df_i, fourier_df_i, ridge_df_i, fig_i = pyBoating(
    time, signal, "detrended", T_sampling, T_cutoff, T_window, periods, pwr_thresh, window_ridge, False, pwr_max)

